### To Do

Fix get_document_path from link_viewer in docket_report

Sometimes extraction doesn't work because context + system prompt is too long

Add manually labelling helper function

Reranking
- https://adasci.org/a-hands-on-guide-to-enhance-rag-with-re-ranking/
- https://techcommunity.microsoft.com/t5/microsoft-developer-community/doing-rag-vector-search-is-not-enough/ba-p/4161073
- https://community.openai.com/t/bad-formats-for-semantic-search-of-rag-implementing-internal-chatbot-for-troubleshooting-an-sdk/848715
- https://learn.microsoft.com/en-us/azure/search/index-similarity-and-scoring
- https://cohere.com/blog/rerank-3
- https://www.reddit.com/r/LocalLLaMA/comments/1d9h2pg/doing_rag_vector_search_is_not_enough/
- https://www.datacamp.com/tutorial/boost-llm-accuracy-retrieval-augmented-generation-rag-reranking
- https://python.langchain.com/v0.2/docs/integrations/retrievers/flashrank-reranker/

RAG
- https://ollama.com/blog/embedding-models
- https://huggingface.co/learn/nlp-course/chapter5/6
- https://docs.mistral.ai/guides/rag/
- https://docs.trychroma.com/guides

In [1]:
import numpy as np
import pandas as pd
import os
from utils import RAG, DocDF
from variable_extractor import VariableExtractor, CaseMetadata, JuryRulingClassifier, CaseDirectory

### Test CaseDirectory 100_random_sample

In [8]:
d = CaseDirectory("100_random_sample")
pd.DataFrame(d.get_metadata_json())

,court,title,docket,judges,judge,type,link,status,flags,nature_of_suit,cause,magistrate,metadata_path
0,"Oregon State, Multnomah County, Circuit Court",State of Oregon vs Abed Alkader Fattoum,18CR08309,"[Angel Lopez, Angela Lucero, Benjamin Souede, ...",NaN,Offense Felony,https://www.docketalarm.com/cases/Oregon_State...,Closed,NaN,NaN,NaN,NaN,100_random_sample/Oregon_State_Multnomah_Count...
1,"Texas State, Harris County, 152nd District Court","CASARES, DOUGLAS vs. CASARES, ROSA",202236481,[ROBERT K. SCHAFFER],ROBERT K. SCHAFFER,Motor Vehicle Accident,https://www.docketalarm.com/cases/Texas_State_...,Disposed (Final),NaN,NaN,NaN,NaN,100_random_sample/Texas_State_Harris_County_15...
2,"Texas State, Dallas County, 101st District Court",ANDREW HOWARD vs. JIMMY SIMPSON,DC-19-03669,[STACI WILLIAMS],STACI WILLIAMS,MOTOR VEHICLE ACCIDENT,https://www.docketalarm.com/cases/Texas_State_...,CLOSED,NaN,NaN,NaN,NaN,100_random_sample/Texas_State_Dallas_County_10...
3,"Massachusetts State, Superior Court, Barnstabl...","Sprague, Dawn vs. Corner Cycle Of Cape Cod, Inc.",2072CV00184,NaN,,Torts,https://www.docketalarm.com/cases/Massachusett...,Open,NaN,NaN,NaN,NaN,100_random_sample/Massachusetts_State_Superior...
4,"Texas State, Dallas County, 116th District Court",DESOTO APARTMENTS LTD vs. DALLAS CENTRAL APPRA...,DC-13-11287,[TONYA PARKER],TONYA PARKER,TAX APPRAISAL,https://www.docketalarm.com/cases/Texas_State_...,CLOSED,NaN,NaN,NaN,NaN,100_random_sample/Texas_State_Dallas_County_11...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
102,"Connecticut State, Superior Court","MACCHIAROLI, ANTHONY v. OFFICE OF CLAIMS COMIS...",LLI-CV23-5015421-S,[ANDREW RORABACK],ANDREW RORABACK,M90 - Misc - All other,https://www.docketalarm.com/cases/Connecticut_...,NaN,NaN,NaN,NaN,NaN,100_random_sample/Connecticut_State_Superior_C...
103,"Connecticut State, Superior Court","DAO, THAO v. ST. LAURENT, GERALD",HHD-CV17-6080452-S,"[CONSTANCE EPSTEIN, DAVID SHERIDAN, SUSAN COBB]",None,V01 - Vehicular - Motor Vehicles - Driver and/...,https://www.docketalarm.com/cases/Connecticut_...,NaN,NaN,NaN,NaN,NaN,100_random_sample/Connecticut_State_Superior_C...
104,"Connecticut State, Superior Court","RENFORS, TARYN v. BRANFORD MANOR PRESERVATION....",KNL-CV21-6051823-S,[KAREN GOODROW],KAREN GOODROW,T03 - Torts - Defective Premises - Private - O...,https://www.docketalarm.com/cases/Connecticut_...,NaN,NaN,NaN,NaN,NaN,100_random_sample/Connecticut_State_Superior_C...
105,"Connecticut State, Superior Court","ANDINO, GIRELYS Et Al v. KAISER, MAYA Et Al",UWY-CV22-6068059-S,[ROBERT D ANDREA],ROBERT D ANDREA,V01 - Vehicular - Motor Vehicles - Driver and/...,https://www.docketalarm.com/cases/Connecticut_...,NaN,NaN,NaN,NaN,NaN,100_random_sample/Connecticut_State_Superior_C...


### Test CaseDirectory 100_random_fed

In [3]:
f = CaseDirectory("100_random_fed")
print(f.get_proportion_downloaded())
print(f.get_mean_downloaded_per_case())

### Test CaseMetadata

In [2]:
m = CaseMetadata.from_path("100_random_sample/Oregon_State_Multnomah_County_Circuit_Court/18CR08309/metadata.json")

In [3]:
dr = m.get_docket_report()

In [4]:
dr

,date,contents,link,link_viewer,title,document_path
0,2021-03-30,Disposition - Reported Created: 03/30/2021 7:3...,NaN,NaN,NaN,
1,2021-03-30,Closed Created: 03/30/2021 2:19 PM,NaN,NaN,NaN,
2,2021-03-30,Judgment - General Eric Dahlin Signed: 03/18/2...,https://www.docketalarm.com/cases/Oregon_State...,https://www.docketalarm.com/cases/Oregon_State...,Judgment - General,100_random_sample/Oregon_State_Multnomah_Count...
3,2021-03-16,CANCELED Call (9:15 AM) Stephen Bushong Judgme...,NaN,NaN,NaN,
4,2021-03-12,Order - Pending Judgment Eric Dahlin Signed: 0...,https://www.docketalarm.com/cases/Oregon_State...,https://www.docketalarm.com/cases/Oregon_State...,Order - Pending Judgment,100_random_sample/Oregon_State_Multnomah_Count...
...,...,...,...,...,...,...
64,2018-02-06,Affidavit - Probable Cause Created: 02/06/2018...,https://www.docketalarm.com/cases/Oregon_State...,https://www.docketalarm.com/cases/Oregon_State...,Affidavit - Probable Cause,100_random_sample/Oregon_State_Multnomah_Count...
65,2018-02-05,Order - Appear Benjamin Souede Signed: 02/05/2...,https://www.docketalarm.com/cases/Oregon_State...,https://www.docketalarm.com/cases/Oregon_State...,Order - Appear,100_random_sample/Oregon_State_Multnomah_Count...
66,2018-02-05,Information Created: 02/06/2018 7:46 AM,https://www.docketalarm.com/cases/Oregon_State...,https://www.docketalarm.com/cases/Oregon_State...,Information,100_random_sample/Oregon_State_Multnomah_Count...
67,2018-02-05,"Arraignment (Judicial Officer: Oden-Orr, Melvi...",NaN,NaN,NaN,


### DF Method

In [20]:
df = pd.read_csv("labeled_cases.csv")

In [21]:
df

,court,case,trial_type,actual_category,keyword_category,mistral_summary,mistral_category,mistral_reasoning,consensus,trial_result,trial_result_reasoning,metadata_path
0,"Oregon State, Multnomah County, Circuit Court",State of Oregon vs Abed Alkader Fattoum,plea,neither,neither,"Defendant Benjamin Souede arraigned twice, Fe...",neither,The case includes documents related to sentenc...,neither,NaN,NaN,workdata/100_random_sample/Oregon_State_Multno...
1,"Texas State, Harris County, 152nd District Court","CASARES, DOUGLAS vs. CASARES, ROSA",order_of_non_suit,neither,jury,Melissa Rosales and Douglas Casares file sepa...,neither,The event 'jury trial is demanded by both part...,neither,NaN,NaN,workdata/100_random_sample/Texas_State_Harris_...
2,"Texas State, Dallas County, 101st District Court",ANDREW HOWARD vs. JIMMY SIMPSON,settlement,neither,jury,Case originally filed as a civil action.\nDef...,jury,The event 'Case #101 began with a jury trial i...,jury,NaN,NaN,workdata/100_random_sample/Texas_State_Dallas_...
3,"Massachusetts State, Superior Court, Barnstabl...","Sprague, Dawn vs. Corner Cycle Of Cape Cod, Inc.",settlement,neither,jury,Plaintiff Paul R. Kelley filed a complaint wi...,neither,The case includes multiple pre-trial conferenc...,neither,NaN,NaN,workdata/100_random_sample/Massachusetts_State...
4,"Texas State, Dallas County, 116th District Court",DESOTO APARTMENTS LTD vs. DALLAS CENTRAL APPRA...,agreed_judgement,neither,jury,"* Original Petition filed, outlining case det...",neither,The event 'A legal dispute between DeSoto Apar...,neither,NaN,NaN,workdata/100_random_sample/Texas_State_Dallas_...
...,...,...,...,...,...,...,...,...,...,...,...,...
102,"Connecticut State, Superior Court","MACCHIAROLI, ANTHONY v. OFFICE OF CLAIMS COMIS...",dismissed,neither,neither,NaN,NaN,NaN,neither,NaN,NaN,workdata/100_random_sample/Connecticut_State_S...
103,"Connecticut State, Superior Court","DAO, THAO v. ST. LAURENT, GERALD",jury,jury,jury,NaN,NaN,NaN,neither,NaN,NaN,workdata/100_random_sample/Connecticut_State_S...
104,"Connecticut State, Superior Court","RENFORS, TARYN v. BRANFORD MANOR PRESERVATION....",withdrawal_of_action,neither,neither,NaN,NaN,NaN,neither,NaN,NaN,workdata/100_random_sample/Connecticut_State_S...
105,"Connecticut State, Superior Court","ANDINO, GIRELYS Et Al v. KAISER, MAYA Et Al",jury,jury,jury,NaN,NaN,NaN,neither,NaN,NaN,workdata/100_random_sample/Connecticut_State_S...


### Test JuryRulingClassifier

In [6]:
path = 'workdata/100_random_sample/Delaware_District_Court/1--21-cv-01238/metadata.json'
classifier = JuryRulingClassifier(path)

In [8]:
classifier.extract()

({},
 {'metadata_response': {'reasoning': "The documents describe motions for summary judgment by both parties and a motion to bifurcate and stay the issue of damages. No verdict sheet is provided in the documents. Therefore, it's not possible to determine from these documents whether the jury ruled in favor of the plaintiff (Rockwell Automation, Inc.) or the defendant (Parcop S.R.L. d/b/a WiAutomation).",
   'category': 'undetermined'},
  'metadata_context': 'MEMORANDUM ORDER: 1. Defendant\'s Motion for Judgment on the Pleadings (D.I. 42) is DENIED AS MOOT. 2. Rockwell\'s Motion for Summary Judgment on Counts I, II, IV, and V (D.I. 144) is DENIED. 3. Defendant\'s Motion for Summary Judgment on All Counts of Rockwell\'s Complaint (D.I. 153) is GRANTED-IN-PART and DENIED-IN-PART as set forth herein. 4. Rockwell\'s Motion to Exclude the opinions of Defendant\'s Expert Justin Blok (D.I. 149) is GRANTED. Mr. Blok is prohibited from testifying to matters related to pp. 16, 29-31, 34-36, 41-